In [1035]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import os

import tensorflow as tf
import keras as keras
from keras.datasets.mnist import load_data
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Conv2D

from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import keras_retinanet as keras_retinanet
from tensorflow.keras.optimizers import Adam
from numpy import reshape




In [1036]:
dataset1 = pd.read_csv("TrainData_x1.csv", sep=';')
dataset1

,Fluid_lvl_tank102,Pressure_tank103,Pump_speed,Flowmeter,Tank102_set,Pressure_set,Flow_set,Fluid_lvl_tank101_IO_link,Fluid_lvl_tank102_IO_link,Fluid_lvl_MAX_tank101,...,V104,V106,V107,V108,V109,V112,Cycle_nuber,EStop,Work,State
0,80.09615,147.431880,0.000000,0.00,0.0,150.0,0.0,105,80,0,...,0,0,0,1,0,0,1,0,0,2
1,79.75961,150.492770,70.675030,0.31,0.0,150.0,0.0,105,80,0,...,0,0,0,1,0,0,1,0,1,2
2,79.75961,150.747830,70.748890,0.00,0.0,150.0,0.0,105,80,0,...,0,0,0,1,0,0,1,0,1,2
3,79.59135,150.747830,70.525444,0.00,0.0,150.0,0.0,105,80,0,...,0,0,0,1,0,0,1,0,1,2
4,79.75961,149.217390,70.285675,0.00,0.0,150.0,0.0,105,80,0,...,0,0,0,1,0,0,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18551,81.27404,7.397102,0.000000,-0.36,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,4,0,1,9
18552,81.10577,7.397102,0.000000,-0.34,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,4,0,1,9
18553,80.60096,7.142029,0.000000,-0.34,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,4,0,1,9
18554,80.43269,7.397102,0.000000,-0.36,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,4,0,1,9


In [1037]:
label = pd.read_csv("TrainData_y1.csv",sep=';')
label

,Failure_F1,Failure_F2,Failure_F3,Failure_F4,Failure_F5
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
18551,0,0,0,0,1
18552,0,0,0,0,1
18553,0,0,0,0,1
18554,0,0,0,0,1


In [1038]:
testdataset = pd.read_csv("TestData_x1.csv",sep=';')
testdataset

,Fluid_lvl_tank102,Pressure_tank103,Pump_speed,Flowmeter,Tank102_set,Pressure_set,Flow_set,Fluid_lvl_tank101_IO_link,Fluid_lvl_tank102_IO_link,Fluid_lvl_MAX_tank101,...,V104,V106,V107,V108,V109,V112,Cycle_nuber,EStop,Work,State
0,79.75961,36.475365,0.000000,-0.35,0.0,150.0,0.0,105,85,0,...,0,0,0,0,0,1,6,0,1,2
1,79.59135,208.394210,80.351440,0.27,0.0,150.0,0.0,105,85,0,...,0,0,0,1,0,0,6,0,1,2
2,79.59135,192.834800,75.014730,0.00,0.0,150.0,0.0,105,85,0,...,0,0,0,1,0,0,6,0,1,2
3,79.59135,165.542040,72.681660,0.00,0.0,150.0,0.0,105,85,0,...,0,0,0,1,0,0,6,0,1,2
4,79.59135,158.400010,71.367195,0.00,0.0,150.0,0.0,105,85,0,...,0,0,0,1,0,0,6,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,81.27404,7.397102,0.000000,-0.34,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,5,0,1,9
2084,81.10577,7.142029,0.000000,-0.36,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,5,0,1,9
2085,80.76923,7.142029,0.000000,-0.35,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,5,0,1,9
2086,80.60096,7.397102,0.000000,-0.35,0.0,0.0,0.0,105,85,0,...,0,0,1,0,0,1,5,0,1,9


In [1039]:
testlabel = pd.read_csv("TestData_y1.csv",sep=';')
testlabel

,Failure_F1,Failure_F2,Failure_F3,Failure_F4,Failure_F5
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
2083,0,0,0,0,1
2084,0,0,0,0,1
2085,0,0,0,0,1
2086,0,0,0,0,1


In [1040]:
x_train = dataset
y_train = label
x_test = testdataset
y_test = testlabel

In [1041]:
#x_train=x_train.to_numpy()
#x_train
#print(x_train.shape)

In [1042]:
#nump_x=x_train.to_numpy()
#nump_y=y_train.to_numpy()

In [1043]:
#tensor_x=torch.from_numpy(nump_x)
#tensor_y=torch.from_numpy(nump_y)
#print(tensor_x.shape)
#print(tensor_y.shape)

In [1044]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)


In [1045]:
x_train = x_train.reshape(x_train.shape[0], 1, 24).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 1, 24).astype('float32') 

In [1046]:
print(x_train.shape)

(18556, 1, 24)


In [1047]:
#x = np.asarray(x_train).astype(np.float32)
#y = np.asarray(y_train).astype(np.float32)


In [1048]:
#Create empty model with your number of classes
#num_classes = 5
#image = keras.layers.Input((None, None, 24))
#model = resnet50_retinanet(num_classes, weights='imagenet', nms=True)


In [1049]:
#Load weights, skipping incompatible sections
#model.load_weights('resnet50_coco_best_v1.2.2.h5', skip_mismatch=True, by_name=True)

In [1050]:
#model = keras.models.Sequential([
#    keras.layers.Conv1D(24, (3), activation='relu', input_shape = [None,24,None]),
#    keras.layers.MaxPooling1D(),
#    keras.layers.Conv1D(16,(2), activation='relu'),
#    keras.layers.MaxPooling1D(),
#    keras.layers.Conv1D(10,(2), activation='relu'),
#    keras.layers.Flatten(),
#    keras.layers.Dense(128, activation='relu'),
#    keras.layers.Dense(5, activation ='softmax')
#])



In [1051]:
model = Sequential()

In [1052]:
model.add(Conv1D(64, (1), activation='relu', input_shape = [1,24]))

In [1053]:
model.add(Conv1D(64,(1), activation='relu'))

In [1054]:
model.add(Dropout(0.2))

In [1055]:
model.add(MaxPooling1D(pool_size=1))

In [1056]:
model.add(Flatten())

In [1057]:
model.add(Dense(100, activation='relu'))

In [1058]:
model.add(Dense(5,activation='softmax'))

In [1059]:
#model = Sequential()
#model.add(LSTM(64, return_sequences = True, input_shape=(0,445344,0), activation='relu'))
#model.add(Dropout(0.1))
#model.add(LSTM(128, activation = 'relu', return_sequences = True))
#model.add(Conv1D(32, kernel_size= 1, input_shape = (0,445344,0), activation = 'relu'))
#model.add(Flatten())
#model.add(Dense(5))

In [1060]:
model.compile(
optimizer= keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07), 
loss= keras.losses.CategoricalCrossentropy(
    from_logits=False,
    #label_smoothing=0.0,
    #axis=-1,
),
metrics= ['accuracy'])

In [1061]:
model.summary()

Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_158 (Conv1D)         (None, 1, 64)             1600      
                                                                 
 conv1d_159 (Conv1D)         (None, 1, 64)             4160      
                                                                 
 dropout_4 (Dropout)         (None, 1, 64)             0         
                                                                 
 max_pooling1d_66 (MaxPoolin  (None, 1, 64)            0         
 g1D)                                                            
                                                                 
 flatten_40 (Flatten)        (None, 64)                0         
                                                                 
 dense_76 (Dense)            (None, 100)               6500      
                                                     

In [1062]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])

d:\DS\Python\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [1063]:
history =model.fit(x_train, y_train, epochs=1000)

Epoch 1/1000
580/580 [==============================] - 2s 2ms/step - loss: 0.9627 - accuracy: 0.5258
Epoch 2/1000
580/580 [==============================] - 1s 2ms/step - loss: 0.8004 - accuracy: 0.5806
Epoch 3/1000
580/580 [==============================] - 1s 2ms/step - loss: 0.7615 - accuracy: 0.5933
Epoch 4/1000
580/580 [==============================] - 1s 2ms/step - loss: 0.7329 - accuracy: 0.6157
Epoch 5/1000
580/580 [==============================] - 1s 2ms/step - loss: 0.7056 - accuracy: 0.6271
Epoch 6/1000
580/580 [==============================] - 1s 2ms/step - loss: 0.6808 - accuracy: 0.6398
Epoch 7/1000
580/580 [==============================] - 1s 2ms/step - loss: 0.6619 - accuracy: 0.6485
Epoch 8/1000
580/580 [==============================] - 1s 2ms/step - loss: 0.6503 - accuracy: 0.6489
Epoch 9/1000
580/580 [==============================] - 1s 2ms/step - loss: 0.6428 - accuracy: 0.6572
Epoch 10/1000
580/580 [==============================] - 1s 2ms/step - loss: 0.640

In [1071]:
model.evaluate(x_test, y_test, verbose=2)


66/66 - 0s - loss: 92.0569 - accuracy: 0.4076 - 103ms/epoch - 2ms/step


[92.05692291259766, 0.4075670540332794]

In [1072]:
metrics = pd.DataFrame(history.history)
metrics

,loss,accuracy
0,0.962662,0.525814
1,0.800358,0.580621
2,0.761454,0.593285
3,0.732885,0.615704
4,0.705635,0.627129
...,...,...
995,0.449043,0.777808
996,0.443415,0.779694
997,0.442637,0.777592
998,0.438396,0.777592


In [1073]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=metrics['loss'], name='loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=metrics['accuracy'], name='accuracy'), row=2, col=1)
fig.update_layout(width=800)

In [1074]:
x_pred=model.predict(x_test)
print(y_test)
print(x_pred)

66/66 [==============================] - 0s 1ms/step
      Failure_F1  Failure_F2  Failure_F3  Failure_F4  Failure_F5
0              1           0           0           0           0
1              1           0           0           0           0
2              1           0           0           0           0
3              1           0           0           0           0
4              1           0           0           0           0
...          ...         ...         ...         ...         ...
2083           0           0           0           0           1
2084           0           0           0           0           1
2085           0           0           0           0           1
2086           0           0           0           0           1
2087           0           0           0           0           1

[2088 rows x 5 columns]
[[2.3578253e-02 0.0000000e+00 9.7642171e-01 0.0000000e+00 0.0000000e+00]
 [2.9499209e-02 1.9103378e-30 0.0000000e+00 7.7038840e-28 9.7050077e-

In [1075]:

y_pred=np.argmax(x_pred,axis=1)
y_pred


array([2, 4, 0, ..., 4, 4, 4], dtype=int64)

In [1076]:
pred= pd.concat([pd.DataFrame(y_test, columns=['Failure_F1','Failure_F2','Failure_F3','Failure_F4','Failure_F5']), pd.DataFrame(y_pred, columns=['y_pred'])], axis=1)
pred.head(10)

,Failure_F1,Failure_F2,Failure_F3,Failure_F4,Failure_F5,y_pred
0,1,0,0,0,0,2
1,1,0,0,0,0,4
2,1,0,0,0,0,0
3,1,0,0,0,0,4
4,1,0,0,0,0,0
5,1,0,0,0,0,0
6,1,0,0,0,0,0
7,1,0,0,0,0,0
8,1,0,0,0,0,0
9,1,0,0,0,0,0


In [1070]:
misclassified = pred[pred['y_test'] != pred['y_pred']]
misclassified.index[:10]

KeyError: 'y_test'